# Part 1: Next-Streamer Prediction

**Given that a user watched Streamer X, what streamer is the user most likely to watch next?**

This question is particularly important for a Twitch-based reccomendation system to be able to accurately predict, and thus reccomend to users, streamers that they would be likely to watch based on their watch history.

In [1]:
# library imports

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import itertools
from math import exp

In [8]:
# Loading in data -> using 100k subset of data for training purposes (full data is >6GB).

path = "data/100k_a.csv"
colnames = ["user_id", "stream_id", "streamer_username", "time_start", "time_stop"]

df = pd.read_csv(path, header=None, names=colnames)
print("Loaded:", df.shape)
df.head()

Loaded: (539934, 5)


,user_id,stream_id,streamer_username,time_start,time_stop
0,1,33842865744,mithrain,154,156.0
1,1,33846768288,alptv,166,169.0
2,1,33886469056,mithrain,587,588.0
3,1,33887624992,wtcn,589,591.0
4,1,33890145056,jrokezftw,591,594.0


In [9]:
# Sort by start time

df = df.sort_values("time_start").reset_index(drop=True)
df.head()

,user_id,stream_id,streamer_username,time_start,time_stop
0,6322,33827512016,stray228,0,6.0
1,14789,33828289776,sayakatcosplay,0,2.0
2,6210,33827512848,chocotaco,0,6.0
3,116,33828483008,scrub,0,2.0
4,4027,33825795040,timthetatman,0,7.0


# Basic EDA

Exploring the dataset we are using.

As we have seen from above, our columns are:

`user_id	| stream_id	| streamer_username	| time_start | time_stop`

In [11]:
# Basic EDA

df.describe()

,user_id,stream_id,time_start,time_stop
count,539934.000000,5.399340e+05,539934.000000,539933.000000
mean,9103.427560,3.412978e+10,3143.541964,3146.698677
std,5191.180986,1.679668e+08,1769.873544,1770.005803
min,1.000000,3.380282e+10,0.000000,1.000000
25%,4689.000000,3.398914e+10,1619.000000,1622.000000
50%,9115.000000,3.413055e+10,3173.000000,3176.000000
75%,13646.000000,3.427294e+10,4665.000000,4669.000000
max,17904.000000,3.441642e+10,6147.000000,6148.000000


In [12]:
# Unique users

df['user_id'].value_counts()

,count
user_id,
17309,309
9308,305
17601,287
7735,269
16173,261
...,...
3871,5
4956,5
25,5


In [14]:
# Unique streamers

df['streamer_username'].value_counts()

,count
streamer_username,
ninja,8760
tfue,7733
shroud,4780
riotgames,3055
nickmercs,2727
...,...
kossolapiy,1
thatsjag,1
iammarshmeow,1


In [15]:
# View counts per unique stream

df[['streamer_username', 'stream_id']].value_counts()

streamer_username  stream_id  
tfue               34265480032    413
ninja              34195155728    395
tfue               34254041488    358
                   34000965936    357
ninja              34401244464    335
                                 ... 
06wallst           34277636448      1
0_doublezero_0     34225419568      1
0akleygames        34361236368      1
0baku              34206197648      1
0blackhound0       34057576832      1
Name: count, Length: 227067, dtype: int64

In [74]:
# Unique counts

df.nunique()

,0
user_id,17904
stream_id,227067
streamer_username,56944
time_start,6148
time_stop,6148


## Evaluation Metrics
We evaluate recommendation quality using two standard metrics for ranking-based recommender systems:

**Hit@K**  
Measures whether the true next streamer appears anywhere in the top-K predictions. It reflects how often the model successfully includes the correct item among its recommendations.

**MRR@K**  
Mean Reciprocal Rank rewards the model for placing the correct streamer near the top of the ranked list. A higher MRR indicates better ordering of recommendations, not just correct inclusion.

---
These metrics are appropriate because next-item prediction is a ranking problem: The model must produce a short list of recommended streamers, and performance depends both on whether the correct streamer is included (Hit@K) and how highly it appears (MRR).

In [42]:
# Metrics

def hit_at_k(preds, true_item, K):
    return int(true_item in preds[:K])

def compute_mrr(preds, true_item):
    for r, item in enumerate(preds, 1):
        if item == true_item:
            return 1/r
    return 0.0

## Baseline Models
We will compare our hybrid recommender against two simple but informative baselines.

**1. Repeat-Last Baseline**  
Always predicts that a user will return to the streamer they just watched.  
This captures strong short-term loyalty and provides a lower bound for sequential models.

**2. Popularity Baseline (Top-10 Streamers)**  
Recommends the top 10 globally most-watched streamers. This measures how well a trivial popularity-based ranking performs, independent of user behavior.

Our goal is to beat out these baselines significantly with our model.


In [46]:
# Baseline 1 — Repeat Last-Watched Streamer by User
def baseline_repeat_last(user, current_streamer, K=10):
    return [current_streamer] * K


# Baseline 2 — Top 10 Most Popular Streamers
TOP10_POPULAR = list(popularity_series.index[:10])
def baseline_popularity(user, current_streamer, K=10):
    return TOP10_POPULAR[:K]


# Evaluate Baselines on Validation Set
def evaluate_baseline(model_fn, df, K=10):
    hits = 0
    mrr_sum = 0

    for row in df.itertuples(index=False):
        preds = model_fn(row.user_id, row.current_streamer, K)
        hits += hit_at_k(preds, row.next_streamer, K)
        mrr_sum += compute_mrr(preds, row.next_streamer)

    return hits/len(df), mrr_sum/len(df)

print("=== Baseline Comparisons (Validation Set) ===")
hit1, mrr1 = evaluate_baseline(baseline_repeat_last, val_df)
print(f"Repeat-Last Baseline: Hit@10={hit1:.4f}, MRR@10={mrr1:.4f}")
hit2, mrr2 = evaluate_baseline(baseline_popularity, val_df)
print(f"Popularity Baseline:  Hit@10={hit2:.4f}, MRR@10={mrr2:.4f}")


=== Baseline Comparisons (Validation Set) ===
Repeat-Last Baseline: Hit@10=0.0769, MRR@10=0.0769
Popularity Baseline:  Hit@10=0.0742, MRR@10=0.0338


## Interpretation of Baseline Results
Both baselines perform poorly, indicating that simple heuristics are not sufficient for next-streamer prediction in this dataset.

- **Repeat-Last (Hit@10 = 0.0769)**  
  Users rarely return to the same streamer in the next viewing interval.  
  This suggests high switching behavior and weak short-term stickiness.

- **Popularity (Hit@10 = 0.0742)**  
  Recommending the top global streamers performs similarly to the repeat-last rule, but with an even lower MRR.  
  This means that the most popular streamers are not strong predictors of what any specific user will watch next.

Overall, these baselines establish a low starting point, confirming that a learned sequential model should be able to outperform simple rules.

## Hybrid Approach Overview

Our approach is to adopt a hybrid recommendation strategy combining:

1. **Long-term user preferences**  
   * how often a user historically transitions into each streamer  
   * captures personalized behavior and consistent favorites

2. **Short-term sequential signals**  
   * transition patterns between streamers
   * co-occurrence of consecutive streamers  
   * captures immediate session-level behavior

3. **Recency weighting**  
   * recent transitions receive higher importance than older ones  
   * adapts the model to fast-changing livestream dynamics

4. **Global popularity smoothing**  
   * incorporates the most common streamers system-wide  
   * stabilizes predictions for sparse users/streamers

5. **Candidate generation**  
   * restricts predictions to a personalized subset instead of the full item space  
   * dramatically improves accuracy and scalability

6. **Hybrid weighted model**  
   * combines all four signals (personal, Markov, co-occurrence, popularity)  
   * weights are tuned using grid search on a time-based validation set

This design reflects modern industry recommender architectures like YouTube's WatchNext, combining personalization, sequential modeling, and recency into a unified ranking model optimized for Hit@K and MRR.


## Building Sequential Transitions
We convert raw event logs into (current_streamer -> next_streamer) transitions by tracking, for each user, the streamer they watched previously and pairing it with the next one in time. This produces a dataset of ordered user viewing steps used by all downstream sequential models.


In [16]:
# Build sequential transitions

last = {}
rows = []
for row in df.itertuples(index=False):
    u = row.user_id
    s = row.streamer_username
    t = row.time_start
    if u in last:
        rows.append((u, last[u], s, t))
    last[u] = s

transitions_df = pd.DataFrame(rows, columns=["user_id", "current_streamer", "next_streamer", "time"])
print("Transitions:", transitions_df.shape)
transitions_df.sample(10)

Transitions: (522030, 4)


,user_id,current_streamer,next_streamer,time
414777,15358,danielfenner,kokiii_,4991
310942,12723,sleyt,fewfort,3842
99486,2182,delordione,tfblade,1384
443850,1333,rossmanngroup,symfuhny,5302
40267,6184,pokimane,tfue,605
104496,10650,nickmercs,ninja,1439
467782,15417,dizzykitten,tsm_viss,5563
92742,12104,rappz1,fntzyzilla,1294
307524,10661,armatvhs,armatvhs,3782
124180,5720,esl_csgo,redshell,1681


## Temporal Train/Validation/Test Split
We split transitions chronologically so that the model is always trained on past data and evaluated on future data. The first 70% of transitions form the training set, the next 10% the validation set, and the final 20% the test set.

To ensure a fair evaluation, we keep only users who appear in the training set so that the model never predicts for unseen users.

In [19]:
# Train/Val/Test split

N = len(transitions_df)
train_end = int(0.7 * N)
val_end   = int(0.8 * N)

train_df = transitions_df.iloc[:train_end]
val_df   = transitions_df.iloc[train_end:val_end]
test_df  = transitions_df.iloc[val_end:]

train_users = set(train_df.user_id)

val_df  = val_df[val_df.user_id.isin(train_users)]
test_df = test_df[test_df.user_id.isin(train_users)]

print("Train/Val/Test shapes:", train_df.shape, val_df.shape, test_df.shape)

Train/Val/Test shapes: (365421, 4) (50223, 4) (98403, 4)


## Recency Weighting
To emphasize recent behavior, each transition is assigned a recency weight. Weights decay exponentially with age, so newer transitions contribute more to the model than older ones. This helps the recommender adapt to fast-changing viewing patterns. This concept is used in indsutry as well.

In [43]:
# Recency weighting

TAU = 1008 # (7-day decay) -> 7 days * 144 units/day (every 10 minutes)
max_t = train_df["time"].max()
train_df.loc[:, "recency_weight"] = np.exp(-(max_t - train_df["time"]) / TAU)
train_df.sample(10)

,user_id,current_streamer,next_streamer,time,recency_weight
303284,1731,solaryfortnite,lestream,3734,0.502333
363685,9811,boraslegend,boraslegend,4410,0.982301
160306,180,solary,laink,2126,0.101903
190671,13990,tsm_hamlinz,xqcow,2467,0.142925
278264,12919,nickslots,spraggy,3457,0.381634
170287,13522,timthetatman,dakotaz,2251,0.115357
267242,12919,kaceytron,smallant1,3332,0.337125
72788,10621,loltyler1,scarra,1041,0.034731
192945,3954,tfue,diegosaurs,2497,0.147242
97833,13784,hootey,shinyzeni,1347,0.047050


## Co-Occurrence Model

We constructed a co-occurrence model that counts how often streamers appear **near each other** in a user's viewing sequence, using a sliding window of size 4. For each user, every pair of streamers occurring within the same short window contributes to a symmetric co-occurrence count.

After aggregating counts across all users, we normalize them into probabilities. This yields a model capturing **session-level association**: streamers commonly watched in the same short viewing window, rather than only immediate next-step transitions.

**Example:**  
`co_probs['ninja']` gives a probability distribution over streamers frequently watched in close proximity to *Ninja*.  
A value like `co_probs['ninja']['tfue'] = 0.0865` means that about **9%** of Ninja's window-based co-occurrences involve Tfue, indicating a moderate association between their viewers.


In [69]:
# Co-occurence

WINDOW_SIZE = 4
co_counts = defaultdict(Counter)

for user, group in train_df.groupby("user_id"):
    seq = list(group.sort_values("time")["next_streamer"])

    for i, A in enumerate(seq):
        for B in seq[i+1 : i + WINDOW_SIZE]:
            if A == B:
                continue
            co_counts[A][B] += 1
            co_counts[B][A] += 1

co_probs = {A: {B: w / sum(c.values()) for B, w in c.items()} for A, c in co_counts.items()}
co_probs['ninja']['tfue']

0.08659326654157963

## Recency-Weighted Markov Model
We constructed a first-order Markov model where each transition  
`current_streamer → next_streamer` is assigned a probability based on how frequently it occurs in users' viewing sequences, with more recent transitions weighted more heavily.

We aggregate recency-weighted counts for every pair of streamers, compute the total outgoing weight for each current streamer, and normalize to obtain: `P(next | current)`

This produces a probability distribution over the likely next streamer conditioned on the current one.

**“Given right now you're watching Ninja, what's the next most likely?”**

For example, `markov['ninja']` represents the **Markov transition probabilities from Ninja**, telling us how likely a user is to watch each possible streamer immediately after watching Ninja, based on recency-weighted viewing behavior across the dataset.


In [70]:
# Markov (recency-weighted)

markov = defaultdict(dict)
g = train_df.groupby(["current_streamer", "next_streamer"])["recency_weight"].sum().reset_index()
g["total"] = g.groupby("current_streamer")["recency_weight"].transform("sum")
g["prob"] = g["recency_weight"] / g["total"]

print(g.shape)
g.head(10)

(241176, 5)


,current_streamer,next_streamer,recency_weight,total,prob
0,007kingchannel,zepher,0.131497,0.131497,1.000000
1,007zeus,berkriptepe,0.018888,0.101935,0.185291
2,007zeus,cerikmeister,0.050826,0.101935,0.498609
3,007zeus,mithrain,0.016438,0.101935,0.161264
4,007zeus,neverlost,0.015783,0.101935,0.154836
5,00flour,metamancer,0.022291,0.022291,1.000000
6,00kys,gamesdonequick,0.215088,0.215088,1.000000
7,01benjamin10,n0rmai,0.029428,0.029428,1.000000
8,06wallst,junkershiryu,0.511891,0.784531,0.652480
9,06wallst,themadmainer,0.272640,0.784531,0.347520


In [67]:
for row in g.itertuples(index=False):
    markov[row.current_streamer][row.next_streamer] = row.prob

markov['ninja']['tfue']

0.05088077243776045

## User Preference Model (recency-weighted)

We build a per-user preference model by counting how often each user watches each streamer, weighted by recency so that more recent interactions have greater influence. For each user, these weighted counts are normalized into a probability distribution over streamers they tend to watch most frequently.

This model captures each user's **long-term personal viewing habits**, complementing the sequence-based models by providing individualized preferences that persist across sessions.

**Example:**  
`user_next_probs[1]` returns the personalized probability distribution over streamers that `user_id = 1` is most likely to watch next, based on their historical, recency-weighted behavior.


In [72]:
# User Preferences (recency-weighted)

user_next = defaultdict(Counter)
for row in train_df.itertuples(index=False):
    user_next[row.user_id][row.next_streamer] += row.recency_weight
user_next_probs = {u: {s: w/sum(c.values()) for s, w in c.items()} for u, c in user_next.items()}
user_next_probs[1]

{'alptv': 0.0016473520002069915,
 'mithrain': 0.24639234184452602,
 'wtcn': 0.0649005276654912,
 'jrokezftw': 0.002511288918108377,
 'berkriptepe': 0.0028940624696532392,
 'kendinemuzisyen': 0.0585919791605092,
 'unlostv': 0.07298622814146097,
 'elraenn': 0.11933960585863232,
 'zeon': 0.1148752182616199,
 'jahrein': 0.05807115972874435,
 'raufbaba25': 0.06508905779050526,
 'ogrencievi': 0.09047933945760775,
 'eraymaskulen': 0.10222183870293441}

## Candidate Generation

To avoid scoring every possible streamer (~57,000 unique streamers), we generated a focused set of candidate streamers for each prediction. We combine the top streamers suggested by several sources:

- **Global popularity** (most-watched streamers overall)  
- **Co-occurrence** (streamers frequently watched in the same session as the current streamer)  
- **Markov transitions** (streamers likely to follow the current streamer in sequence)  
- **User preferences** (streamers the user frequently watches)

The union of these sources forms a compact, high-quality candidate pool. This significantly reduces computation while ensuring that the final ranking model only considers plausible next-streamer predictions.

In [76]:
# Candidate generation

def generate_candidates(user, s_now, top_n=50):
    cands = set(TOP_GLOBAL)
    if s_now in co_probs:
        cands.update(dict(sorted(co_probs[s_now].items(), key=lambda x:-x[1])[:top_n]))
    if s_now in markov:
        cands.update(dict(sorted(markov[s_now].items(), key=lambda x:-x[1])[:top_n]))
    if user in user_next_probs:
        cands.update(dict(sorted(user_next_probs[user].items(), key=lambda x:-x[1])[:top_n]))
    return list(cands)

generate_candidates('1', 'elraenn')[:10]

['drlupo',
 'timthetatman',
 'squeezielive',
 'nazframbuaz',
 'fortnite',
 'pow3rtv',
 'methodjosh',
 'disguisedtoast',
 'tsm_hamlinz',
 'admiralbulldog']

## Hybrid Predictor

The hybrid model combines multiple recommendation signals into a single scoring function. For each candidate streamer, we compute a weighted sum of:

- **User preference score** (how strongly the user has preferred to watched this streamer in the past)  
- **Markov transition score** (likelihood of this streamer following the current one)  
- **Co-occurrence score** (how often this streamer appears in the same session window as the current one)  
- **Global popularity** (overall viewing frequency across the platform)

The weights `(w_u, w_m, w_c, w_p)` control the contribution of each signal. The model ranks all candidate streamers by their combined score and returns the top-K as the final recommendations.


In [79]:
# Hybrid predictor

def predict_hybrid(user, s_now, wu, wm, wc, wp, K=10):
    candidates = generate_candidates(user, s_now)
    scores = Counter()

    for nxt in candidates:
        scores[nxt] += wu * user_next_probs.get(user, {}).get(nxt, 0)
        scores[nxt] += wm * markov.get(s_now, {}).get(nxt, 0)
        scores[nxt] += wc * co_probs.get(s_now, {}).get(nxt, 0)
        scores[nxt] += wp * popularity_series.get(nxt, 0)
    return [s for s,_ in scores.most_common(K)]

predict_hybrid('1', 'elraenn', 0.2, 0.3, 0.4, 0.1)

['kendinemuzisyen',
 'wtcn',
 'mithrain',
 'jahrein',
 'elwind',
 'jrokezftw',
 'zeon',
 'videoyun',
 'grimnax',
 'uthenera']

## Evaluation Function

The `evaluate` function measures model performance using two standard ranking metrics

- **Hit@K** — whether the true next streamer appears anywhere in the model's top-K predictions.
- **MRR@K** — the mean reciprocal rank, which rewards placing the correct streamer higher in the ranked list.

For each row in the evaluation set, we generate predictions, compute both metrics, and return their averages. This provides a clear assessment of how well a model performs on next-streamer prediction.

In [ ]:
# Evaluation Function

def evaluate(model_fn, df, K=10):
    hits = 0
    mrr_sum = 0
    for row in df.itertuples(index=False):
        preds = model_fn(row.user_id, row.current_streamer, K)
        hits += hit_at_k(preds, row.next_streamer, K)
        mrr_sum += compute_mrr(preds, row.next_streamer)
    return hits/len(df), mrr_sum/len(df)

## Weight Grid Search (One-Time Tuning)

To determine the best combination of weights for the hybrid model, we performed a grid search over a small set of candidate values for each component weight. For every combination of `(w_u, w_m, w_c, w_p)`, we evaluate the hybrid model on the validation set and record the Hit@10 score.

This search identifies the weight configuration that produces the strongest next-streamer predictions. Because the full grid can be expensive to compute, it is run only once (on GPU), and the resulting optimal weights are reused for all final experiments.


In [80]:
# # Weight Grid Search -> Commented out due to time to run (ONE-TIME RUN)

# weight_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# best_score = 0
# best_weights = None
# results = {}

# for wu in weight_values:
#     for wm in weight_values:
#         for wc in weight_values:
#             for wp in weight_values:

#                 hit, _ = evaluate(lambda u, s, K: predict_hybrid(u, s, wu, wm, wc, wp, K), val_df, K=10)

#                 results[(wu, wm, wc, wp)] = hit

#                 if hit > best_score:
#                     best_score = hit
#                     best_weights = (wu, wm, wc, wp)
#                     print(f"New best Hit@10={hit:.4f} with (wu={wu}, wm={wm}, wc={wc}, wp={wp})")


# print("\nBest weights:", best_weights)
# print("Best validation Hit@10:", best_score)

## Final Evaluation on the Test Set

After selecting the best-performing weights from the validation search, we apply the hybrid model using these optimized weight values to the held-out test set. This provides an unbiased estimate of the model's true predictive performance.

We report the final **Hit@10** and **MRR@10**, which summarize how effectively the tuned hybrid recommender predicts the next streamer a user will watch in real-world conditions.


In [82]:
# Final Evaluation on Test Set

# From GPU-ran Grid-Search

wu = 0.60
wc = 0.20
wm = 0.15
wp = 0.05
best_weights = (wu, wm, wc, wp)

test_hit, test_mrr = evaluate(lambda u, s, K: predict_hybrid(u, s, wu, wm, wc, wp, K=10), test_df)
print("\nBest Weights:", best_weights)
print("Final Test Hit@10:", test_hit)
print("Final Test MRR@10:", test_mrr)


Best Weights: (0.6, 0.15, 0.2, 0.05)
Final Test Hit@10: 0.4358302084286048
Final Test MRR@10: 0.20348587981816849


## Baseline and Hybrid Model Comparison (Test Set)

We compare the tuned hybrid recommender against two simple baselines to contextualize its performance:

- **Repeat-Last Baseline:** always predicts that the next streamer will be the one the user just watched.
- **Popularity Baseline:** recommends the globally most popular streamers.

We then report the hybrid model’s final Hit@10 and MRR@10 scores on the held-out test set. This comparison highlights how much predictive power the hybrid model gains over naive heuristics.


In [84]:
print("\n=== Baseline Comparisons (Test Set) ===")
hit1, mrr1 = evaluate_baseline(baseline_repeat_last, test_df)
print(f"Repeat-Last Baseline: Hit@10={hit1:.4f}, MRR@10={mrr1:.4f}")
hit2, mrr2 = evaluate_baseline(baseline_popularity, test_df)
print(f"Popularity Baseline:  Hit@10={hit2:.4f}, MRR@10={mrr2:.4f}")

print()

print("\n=== Hybrid Model Results (Test Set) ===")
print(f"Hybrid Model: Hit@10={test_hit:.4f}, MRR@10={test_mrr:.4f}")



=== Baseline Comparisons (Test Set) ===
Repeat-Last Baseline: Hit@10=0.0838, MRR@10=0.0838
Popularity Baseline:  Hit@10=0.0714, MRR@10=0.0298


=== Hybrid Model Results (Test Set) ===
Hybrid Model: Hit@10=0.4358, MRR@10=0.2035


# Results

Our hybrid recommender achieves **Hit@10 = 0.4358** and **MRR@10 = 0.2035**, substantially outperforming the baselines. This shows that combining user preferences, sequential patterns, and co-occurrence signals yields far better predictions than simple heuristics.

---

A **Hit@10 of 0.4358** means that in **~44% of cases, the model includes the correct next streamer somewhere in its top 10 suggestions**. The **MRR@10** score indicates that when the model is correct, it typically ranks the right streamer within the **top few positions**.

---

These results are notable given our constraints: we only used viewing sequences and did **not** incorporate or have access to more advanced features like streamer categories, social trends, timestamps beyond ordering, or user demographics. By contrast, industry systems like YouTube's recommender rely on hundreds of contextual and content-based features and typically achieve significantly higher accuracy. Even so, our hybrid model captures meaningful behavioral structure and clearly improves over naive baselines.

---

Major commercial platforms (YouTube, Netflix, TikTok, Spotify) do not publish Hit@K metrics, so direct comparison is impossible. Academic literature using public datasets typically sees Hit@10 values between 0.40-0.75 for advanced sequence models. Our model's Hit@10 of 0.4358 is consistent with the performance of mid-tier classical recommenders despite having far less contextual and content-based information than industry systems.
